In [3]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
  # ソースコードを記述
  # LLMを定義する
  llm = ChatOpenAI(model=model_name)

  # ツールを定義する（Tavilyの定義）
  tool = TavilySearchResults(max_results=2)
  tools = [tool]
  #LLMとツールを紐づける
  llm_with_tools = llm.bind_tools(tools)

  # チャットボットの処理を定義する
  def chatbot(state: State):
    ai_msg = llm_with_tools.invoke(state["messages"])
    return {"messages": [ai_msg]}

  # 会話の流れを構築してチャットボットのノードを登録する
  graph_builder = StateGraph(State)
  graph_builder.add_node("chatbot", chatbot)

  # ツール（検索）を実行するノードを登録する
  tool_node = ToolNode(tools)
  graph_builder.add_node("tools", tool_node)

  # 分岐の条件付けをする
  graph_builder.add_conditional_edges("chatbot", tools_condition)

  # ツール実行後に、ツールからチャットボットへ戻す
  graph_builder.add_edge("tools", "chatbot")

  # 開始ノードを指定する
  graph_builder.set_entry_point("chatbot")

  # 会話の履歴を持たせる
  memory = MemorySaver()
  return graph_builder.compile(checkpointer=memory)

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
  # ソースコードを記述
  # graph.stream(...) にユーザーの発言を渡すと、会話の各ステップごとに結果が返ってくるようにする
  events = graph.stream(
    {"messages": [("user", user_input)]}, #今回のユーザー発言
    {"configurable": {"thread_id": "1"}}, #会話IDが同じIDなら履歴を引き継ぎされる
    stream_mode="values" # メッセージ(values）を順に受け取る
  )
  for event in events:
    print(event["messages"][-1].content, flush=True) #順番に表示させる

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
  user_input = input("質問: ")
  if user_input.strip() == "":
    print("ありがとうございました!")
    break
  stream_graph_updates(graph, user_input)

こんにちは
こんにちは！今日はどんなことをお手伝いできますか？
今日の横浜市の天気は？

[{"url": "https://tenki.jp/forecast/3/17/4610/14100/10days.html", "content": "éæ°´é\n    :   2㎜ 0㎜ 4㎜ 12㎜\n\n    æ°æ¸©\n\n    æ¹¿åº¦\n    :   80% 78% 65% 74% 81%\n\n    é¢¨\n    :   3m/s\n\n        3m/s\n\n        4m/s\n\n        5m/s\n\n        7m/s\n\næ¥ä» å¤©æ° æ°æ¸© éæ°´ç¢ºç ä¿¡é ¼åº¦\n:   10æ03æ¥(é)\n\n    ææãé¨\n\n    23℃19℃\n\n    80%\n\n    C\n:   10æ04æ¥(å)\n\n    æ\n\n    24℃19℃\n\n    40%\n\n    E\n:   10æ05æ¥(æ¥)\n\n    ææãé¨\n\n    24℃19℃\n\n    80%\n\n    C\n\nAãEã¯å¤©æ°äºå ±ã®ä¿¡é ¼åº¦ã§ã\n\nå¤©æ°äºå ±ã®ä¿¡é ¼åº¦ã®è¦æ¹ [...] æ¥ã®åº/å¥\n    :   æ¥ã®åºï½05:35\n\n        æ¥ã®å¥ï½17:27\n\n    æå»\n    :   00 06 12 18 24\n\n    å¤©æ°\n\n    éæ°´ç¢ºç\n    :   30% 30% 30% 30%\n\n    éæ°´é\n    :   0㎜ 0㎜ 0㎜ 0㎜\n\n    æ°æ¸©\n\n    æ¹¿åº¦\n    :   65% 65% 52% 64% 78%\n\n    é¢¨\n    :   4m/s\n\n        3m/s\n\n        4m/s\n\n        3m/s\n\n        2m/s\